# Pipeline do Modelo

Neste arquivo é feita a construção da pipeline do modelo e exportação do arquivo serial do mesmo.

Importação das bibliotecas necessárias

In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef

/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


Leitura dos dados.

In [2]:
dados = pd.read_csv('desafio_manutencao_preditiva_treino.csv')

Função para preparação dos dados.

In [3]:
def prep_dados(dados):   
    dados.drop(['udi', 'product_id'], axis=1, inplace=True)
    dados = dados.drop(columns='air_temperature_k')
    X = dados.drop(columns='failure_type') # Variáveis independentes
    y = dados[['failure_type']] # Variável dependente
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=.3, stratify=y, random_state=101)
    X_treino = pd.get_dummies(X_treino)
    X_teste = pd.get_dummies(X_teste)
    encoder = LabelEncoder()
    encoder.fit(y_teste)
    y_teste = encoder.transform(y_teste)
    y_treino = encoder.transform(y_treino)
    return X_treino, X_teste, y_treino, y_teste
    

In [4]:
X_treino, X_teste, y_treino, y_teste = prep_dados(dados) # Separação dos conjuntos de treino e teste.

/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Pipeline do modelo

In [5]:
modelo = Pipeline([
    ('scaler', MinMaxScaler()),
    ('model', XGBClassifier())
])

Treinamento do modelo

In [6]:
modelo.fit(X_treino, y_treino)

/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:00:12] WARNING: /croot/xgboost-split_1675119646044/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('scaler', MinMaxScaler()),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, enable_categorical=False,
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1,
                               objective='multi:softprob', predictor='auto',
                               random_state=0, reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=None, subsample=1,
                               tree_method='exact', validate_parameters=1,
                               verbosity=None))])

Testando o modelo

In [7]:
y_predito = modelo.predict(X_teste)

In [8]:
acuracia = accuracy_score(y_teste, y_predito)
precisao = precision_score(y_teste, y_predito, average='weighted')
recall = recall_score(y_teste, y_predito, average='weighted')
f1s = f1_score(y_teste, y_predito, average='weighted')
MCC = matthews_corrcoef(y_teste, y_predito)

print("Acurácia: "+ "{:.2%}".format(acuracia))
print("Recall: "+ "{:.2%}".format(recall))
print("Precisão: "+ "{:.2%}".format(precisao))
print("F1-Score: "+ "{:.2%}".format(f1s))
print("MCC: "+ "{:.2%}".format(MCC))

Acurácia: 97.65%
Recall: 97.65%
Precisão: 96.91%
F1-Score: 97.04%
MCC: 58.53%


/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Confirma-se o desempenho obtido anteriormente.

Exportando o serial do modelo.

In [9]:
import joblib

joblib.dump(modelo, 'modelo_pred_falhas.pkl')


['modelo_pred_falhas.pkl']

Exportando o label encoder

In [10]:
dados = pd.read_csv('desafio_manutencao_preditiva_treino.csv')
dados.drop(['udi', 'product_id'], axis=1, inplace=True)
dados = dados.drop(columns='air_temperature_k')
X = dados.drop(columns='failure_type') # Variáveis independentes
y = dados[['failure_type']] # Variável dependente
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=.3, stratify=y, random_state=101)
X_treino = pd.get_dummies(X_treino)
X_teste = pd.get_dummies(X_teste)
encoder = LabelEncoder()
encoder.fit(y_teste)
encoder = LabelEncoder()
encoder.fit(y_teste)
joblib.dump(encoder, 'label_encoder.pkl')

/home/duartejr/anaconda3/envs/indicium/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['label_encoder.pkl']